### Model PyOD AutoEncoder

In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_auc_score

from imblearn.under_sampling import RandomUnderSampler
from pyod.models.auto_encoder import AutoEncoder

In [2]:
# RESTART POINT

orig_train = pd.read_parquet("train_augmented.parquet")

In [3]:
# handle NaN
orig_train.fillna(value=0, inplace=True)

orig_train.isna().sum()

ip                          0
app                         0
device                      0
os                          0
channel                     0
click_time                  0
is_attributed               0
hour                        0
day                         0
ip_app_channel_var_day      0
qty                         0
ip_app_count                0
ip_app_os_count             0
qty_var                     0
ip_app_os_var               0
ip_app_channel_mean_hour    0
dtype: int64

In [4]:
TARGET = 'is_attributed'

# train, validation split
TRAIN_FRAC = 0.98

N_TRAIN = int(orig_train.shape[0] * TRAIN_FRAC)

# shuffle
orig_train = orig_train.sample(frac=1., random_state=12345)

df_train = orig_train[:N_TRAIN]
df_valid = orig_train[N_TRAIN:]

print('Original training samples:', df_train.shape[0])
print('Num. validation samples:', df_valid.shape[0])

Original training samples: 181205812
Num. validation samples: 3698078


In [6]:
feat = ['ip', 'app', 'device', 'os', 'channel',
       'hour', 'day', 'ip_app_channel_var_day', 'qty', 'ip_app_count',
       'qty_var', 'ip_app_os_count', 'ip_app_os_var',
       'ip_app_channel_mean_hour']

# removed ip because it is high card

cat_feat = ['app', 'device', 'os', 'channel']

x_train = df_train[feat]
y_train = df_train[TARGET]

x_valid = df_valid[feat]
y_valid = df_valid[TARGET]

rus = RandomUnderSampler(random_state=0, sampling_strategy=0.1)

x_train_resampled, y_train_resampled = rus.fit_resample(x_train, y_train)